In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import TransformedTargetRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.tree import ExtraTreeRegressor

import warnings
warnings.filterwarnings(action='ignore')

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [18]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.sample(10)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
617,25/11/2020 14:55:16,42.0,Male,Wroclaw,Software Engineer,16,NaN,Senior,java/scala/go/clouds/devops,"Java / Scala, Go, AWS, Google Cloud, Kubernete...",36000.0,NaN,30,Full-time employee,Unlimited contract,English,1000+,Product
1029,01/12/2020 13:56:36,35.0,Male,Berlin,Software Engineer,20,5,Senior,JavaScript,"Javascript / Typescript, AWS, Kubernetes, Docker",100000.0,0,32,Full-time employee,Unlimited contract,English,1000+,Product
593,25/11/2020 13:58:38,30.0,Male,Berlin,Data Scientist,6,3,Senior,NaN,"Python, SQL, AWS, Google Cloud, Docker",72000.0,85000,27,Full-time employee,Unlimited contract,English,1000+,Startup
544,25/11/2020 11:37:31,29.0,Male,Milan,Data Scientist,4,0,Middle,Python + SQL,"Python, SQL, Spark",37500.0,0,16,Full-time employee,Unlimited contract,Italian,101-1000,Consulting / Agency
793,26/11/2020 21:13:28,32.0,Male,Berlin,Senior Data Engineer,9,4,Senior,Python,"Java / Scala, SQL, AWS",90000.0,NaN,30,Full-time employee,Unlimited contract,English,51-100,Startup
1045,01/12/2020 22:23:17,36.0,Male,Munich,Software Engineer,13,5,Head,Java,"Javascript / Typescript, Java / Scala, AWS, Az...",90000.0,5000,30,Full-time employee,Unlimited contract,English,101-1000,Product
310,24/11/2020 16:16:41,34.0,Female,Berlin,Software Engineer,9,3,Senior,Kotlin,"Python, Kotlin, Java / Scala, Google Cloud, Ku...",78000.0,3000,30,Full-time employee,Unlimited contract,English,101-1000,Product
1075,03/12/2020 21:25:35,38.0,Male,Berlin,Backend Developer,15,1,Middle,PHP/MySQL,"PHP, SQL, Docker, VMs",40700.0,NaN,25,Full-time employee,Unlimited contract,English,11-50,Consulting / Agency
911,29/11/2020 18:41:38,38.0,Male,Berlin,Software Engineer,12,6,Senior,Java,Kotlin,92000.0,40000,Unlimited,Full-time employee,Unlimited contract,English,1000+,Product
7,24/11/2020 11:16:44,24.0,Male,Berlin,Frontend Developer,5,1,Senior,Typescript,Javascript / Typescript,65000.0,NaN,27,Full-time employee,Unlimited contract,English,1000+,Product


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 16 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Age                                                     1226 non-null   float64
 1   Gender                                                  1243 non-null   object 
 2   City                                                    1253 non-null   object 
 3   Position                                                1247 non-null   object 
 4   Total years of experience                               1237 non-null   object 
 5   Years of experience in Germany                          1221 non-null   object 
 6   Seniority level                                         1241 non-null   object 
 7   Your main technology / programming language             1126 non-null   object 
 8   Other technologies/programming languag

In [20]:
#
df["Other technologies/programming languages you use often"].value_counts()

Javascript / Typescript                                                                   44
Python                                                                                    37
SQL                                                                                       31
AWS, Docker                                                                               16
Kotlin                                                                                    15
                                                                                          ..
Python, Ruby, Java / Scala, Go, Rust, AWS, Kubernetes, Docker, Terraform, helm, packer     1
Kotlin, SQL, AWS, Kubernetes, Docker                                                       1
Python, R, Go                                                                              1
C/C++, PHP, SQL                                                                            1
Python, AWS, Kubernetes, Docker, terraform, ansible                   

In [21]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


In [23]:
#Creating target
df["Yearly bonus + stocks in EUR"] = pd.to_numeric(df["Yearly bonus + stocks in EUR"],downcast="float",errors="coerce")
df["Yearly bonus + stocks in EUR"].fillna(0, inplace=True)
df["TotalCompensation"]= df['Yearly brutto salary (without bonus and stocks) in EUR'] + df['Yearly bonus + stocks in EUR']


In [24]:

df.drop(columns={"Yearly bonus + stocks in EUR","Yearly brutto salary (without bonus and stocks) in EUR","Timestamp"},inplace=True)
df.head()

,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type,TotalCompensation
0,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",30,Full-time employee,Unlimited contract,English,51-100,Product,85000.0
1,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product,80000.0
2,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",30,Self-employed (freelancer),Temporary contract,English,101-1000,Product,240000.0
3,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup,54000.0
4,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",29,Full-time employee,Unlimited contract,English,101-1000,Product,62000.0


Filtering outliers

In [ ]:
#df["TotalCompensation"].describe()
df=df[df["TotalCompensation"]<1.000000e+05]
df["TotalCompensation"].describe()

count      990.000000
mean     67331.865788
std      16367.391820
min      10001.000000
25%      57300.000000
50%      69100.000000
75%      80000.000000
max      99000.000000
Name: TotalCompensation, dtype: float64

In [25]:
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() # Specific column 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp

Categorizing less frequent values

In [16]:
df["Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10, "Other")
df["Your main technology / programming language"].replace({"Javascript": "JavaScript", "Python ":"Python"}, inplace=True)
df["Your main technology / programming language"].value_counts()

Other         377
Python        185
Java          184
JavaScript     65
PHP            56
C++            38
C#             30
python         26
Swift          24
Scala          24
Go             23
Kotlin         22
Ruby           19
TypeScript     14
SQL            14
.NET           13
JS             12
Name: Your main technology / programming language, dtype: int64

In [39]:
df["Other technologies/programming languages you use often"].replace({"Javascript": "JavaScript", "Python ":"Python"}, inplace=True)
df["Other technologies/programming languages you use often"].replace({'typescript':'TypeScript','Typescript':'TypeScript','javascript':'JavaScript'},inplace=True)
df["Other technologies/programming languages you use often"].replace({'Php':'PHP','php':'PHP','Javascript / Typescript': 'TypeScript','JS':'JavaScript'},inplace=True)
df["Other technologies/programming languages you use often"] = replace_low_freq(df, "Other technologies/programming languages you use often", 10, "Other")
df["Other technologies/programming languages you use often"].value_counts()

Other          722
Unknown        129
TypeScript      39
Python          30
SQL             25
AWS, Docker     12
Kotlin          11
Swift           11
Python, SQL     11
Name: Other technologies/programming languages you use often, dtype: int64

In [66]:
df["City"] = replace_low_freq(df, "City", 10, "Other")
df["Company type"] = replace_low_freq(df, "Company type", 3, "Other")
df["Position "] = replace_low_freq(df, "Position ", 5, "Other")

In [26]:
df.isna().sum()

Age                                                        27
Gender                                                     10
City                                                        0
Position                                                    6
Total years of experience                                  16
Years of experience in Germany                             32
Seniority level                                            12
Your main technology / programming language               127
Other technologies/programming languages you use often    157
Number of vacation days                                    68
Employment status                                          17
Сontract duration                                          29
Main language at work                                      16
Company size                                               18
Company type                                               25
TotalCompensation                                           0
dtype: i

Imputing missing values with median value

In [40]:
df['Age'].fillna(df['Age'].median(), inplace=True)

df['Total years of experience'] = pd.to_numeric(df['Total years of experience'].astype(str).str.replace(',',''), downcast="float", errors="coerce")
df['Total years of experience'].fillna(df['Years of experience in Germany'].median(),inplace=True)

df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'].astype(str).str.replace(',',''), downcast="float", errors="coerce")
df['Years of experience in Germany'].fillna(df['Years of experience in Germany'].median(),inplace=True)

df['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'].astype(str).str.replace(',',''), downcast="float", errors="coerce")
df['Number of vacation days'].fillna(df['Number of vacation days'].median(),inplace=True)


In [29]:
df['Gender'].fillna('Unknown',inplace=True)
df['Position '].fillna('Unknown',inplace=True)
df['Seniority level'].fillna('Unknown',inplace=True)
df["Your main technology / programming language"].fillna('Unknown',inplace=True)
df['Other technologies/programming languages you use often'].fillna('Unknown',inplace=True)
df['Employment status'].fillna('Unknown',inplace=True)
df['Сontract duration'].fillna('Unknown',inplace=True)
df['Main language at work'].fillna('Unknown',inplace=True)
df['Company size'].fillna('Unknown',inplace=True)
df['Company type'].fillna('Unknown',inplace=True)


In [70]:
# Train and test data split
# Creating pipeline
y= df["TotalCompensation"]
X = df.drop(columns={"TotalCompensation"})
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25)

numeric_features = ['Age','Total years of experience','Years of experience in Germany','Number of vacation days']
numeric_transformer = Pipeline( steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", MinMaxScaler())
    ])

categorical_features = ['Gender','Position ','Seniority level','Your main technology / programming language',
                        'Other technologies/programming languages you use often','Employment status','Сontract duration',
                        'Main language at work','Company size','Company type']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer( 
        transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

param_grid = {
    'criterion':['squared_error','friedman_mse', 'poisson'],
    'min_samples_leaf': [2,3,4],
    'max_features': ['sqrt', 'log2'],
    'max_depth':[3,4,5],
    'ccp_alpha':[0.1,0.5]}
pipeline_grid= [('pre', preprocessor),('search', GridSearchCV(estimator=ExtraTreeRegressor(),n_jobs=-1,param_grid=param_grid))]
grid = Pipeline(pipeline_grid)

In [72]:
# Train the model
grid.fit(X_train, y_train.ravel())
# Evaluate the model on the test set
testscore = grid.score(X_test, y_test.ravel())
trainscore= grid.score(X_train, y_train.ravel())
print(f"Test score: {testscore:.3f}")
print(f"Train score:{trainscore:.3f}")
y_pred = grid.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print('Test RMSE:', rmse)

Test score: 0.067
Train score:0.177
Test RMSE: 16165.625258664559


In [69]:
#Feature selection using Recursive feature elimination
from sklearn.feature_selection import RFE
est = GradientBoostingRegressor()
rfe = RFE(estimator=est, n_features_to_select=25, step=1)
x_reg_dum = pd.get_dummies(X, drop_first=True)
rfe.fit(x_reg_dum, y)
print(rfe.ranking_)
mask = rfe.get_support()
new_features = x_reg_dum.columns[mask]
print(new_features)
Xreg_train, Xreg_test, yreg_train, yreg_test = train_test_split(x_reg_dum, y)
print("Test Score after feature Selection using RFE",rfe.score(Xreg_test,yreg_test))
yreg_pred = rfe.predict(Xreg_test)
reg_mse = mean_squared_error(yreg_test, yreg_pred)
reg_rmse = np.sqrt(mse)

print('test regression RMSE:', reg_rmse)

[  1   1   1   1   1  73  46  72  18   1  78  87  85  74  92  91  84  99
 102   1   3  47 114 112  97  98  66 106 120 111 130 100 104 128 119 109
 116  38  16 122 125  24 149 134 136 124 133 138 143 142 148 150 152 154
 156 159  40 164 169  32 139 161  54  20 163 180  13 181  76  10 183 191
   1   1 200 201 211 198 204 205 208   4 215 194 210 219  69  55  49 230
 227 233  68 235 240 243 242  96 246 231 250 254 252 255 261   5 263  21
  45 247 271 212 273  12 280 283 278 282  17 291 293 287 290 298 295 275
 304 307 306 301 310 316 312 302 317 322   1 325 327 329   1  57 331 324
 314 342 343  60   8 351 378  33 375 353 350 380 358 362 368 382  15   9
  58 336 338 370 372 340 341 391  37 395  67   1 365 402 404 409 400 408
 389 412 428 416 417 414 396 406 426 431 369 430 419 439 418 442  70 383
 394 363 444 423 422 460 462 454  35 451 470 468 472 473 455 464 491 425
 493 495 101 450 447 449   1 467   1   1 421  50  65   1 188  28 216 220
 222 224 236 256 258 264 266 268 284 318 344 346 35

# Answers and Explainations
(Expand/modify as needed)

### Results
Extra tree regressor seems to be the best model with RMSE of 14475.42, Testing score of 0.164 and Training score of 0.141

### Feature Selection Activities
Feature selection is done using recursive feature elimination using gradient boosting regressor  and the accuracy increased by decreasing features.
### Hyperparameter Changes
     criterion:squared_error,friedman_mse, poisson
     Mimimum samples leaf: 2,3,4
     max_features: 'sqrt', 'log2'
     max_depth: 3,4,5
    ccp_alpha:0.1,0.5
    Hyperparameters are tested and modified to increase accuracy